Установка Spark в Windows: https://phoenixnap.com/kb/install-spark-on-windows-10

In [1]:
from pyspark.sql import SparkSession
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" # без этой строчки у нас будет возникать постоянное предупреждение с просьбой установить эту переменную в значение 1, что мы заранее и делаем


spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark

In [1]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"


builder = SparkSession.builder.appName("Delta Lake Spark") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \


spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
data = spark.read.parquet("groups/groups.parquet")
data.write.format("delta").save("delta_table")#, mode="overwrite")

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Hudi Spark") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.jars.packages", "org.apache.hudi:hudi-spark3.4-bundle_2.12:0.14.1") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension") \
    .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar") \
    .getOrCreate()

In [2]:
data = spark.read.parquet("groups/groups.parquet")
data.write.format("hudi").options(**{"hoodie.table.name": "groups"}).save("hudi_catalog", mode="overwrite")

Py4JJavaError: An error occurred while calling o45.save.
: java.lang.RuntimeException: java.net.URISyntaxException: Relative path in absolute URI: file:hudi_catalog/.hoodie
	at org.apache.hudi.common.fs.HoodieWrapperFileSystem.convertPathWithScheme(HoodieWrapperFileSystem.java:164)
	at org.apache.hudi.common.fs.HoodieWrapperFileSystem.convertToDefaultPath(HoodieWrapperFileSystem.java:969)
	at org.apache.hudi.common.fs.HoodieWrapperFileSystem.lambda$getFileStatus$17(HoodieWrapperFileSystem.java:406)
	at org.apache.hudi.common.fs.HoodieWrapperFileSystem.executeFuncWithTimeMetrics(HoodieWrapperFileSystem.java:114)
	at org.apache.hudi.common.fs.HoodieWrapperFileSystem.getFileStatus(HoodieWrapperFileSystem.java:404)
	at org.apache.hudi.exception.TableNotFoundException.checkTableValidity(TableNotFoundException.java:51)
	at org.apache.hudi.common.table.HoodieTableMetaClient.<init>(HoodieTableMetaClient.java:140)
	at org.apache.hudi.common.table.HoodieTableMetaClient.newMetaClient(Hood

In [24]:
from deltalake import DeltaTable, write_deltalake
from pyarrow import Table, parquet, feather


df: Table = parquet.read_table("groups/groups.parquet")
feather.write_feather(df, "groups/groups.feather")

In [29]:
feather_data = feather.read_feather("groups/groups.feather")
write_deltalake("./feather_table", feather_data, mode="append")

In [31]:
dt = DeltaTable("/feather_table")

df = dt.to_pandas()



,Название,Специальность
0,ИАБО-01-04,Информационные технологии и разработка програм...
1,ИАБО-02-04,Сетевые технологии и администрирование сетей
2,ИАБО-03-04,Программирование и разработка на языке Java
3,ИАБО-04-04,Аналитика данных в бизнесе и финансах
4,ИАБО-05-04,Сетевые технологии и администрирование сетей
...,...,...
199995,ИАБО-96-24,Программирование и разработка на языке Java
199996,ИАБО-97-24,Аналитика данных в бизнесе и финансах
199997,ИАБО-98-24,Разработка мобильных приложений и IoT
199998,ИАБО-99-24,Разработка мобильных приложений и IoT


: 